In [15]:
import matplotlib.pyplot as plt
import cv2
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Flatten, Conv2D, MaxPool2D, Dropout
from tensorflow.keras.optimizers import SGD , Adam
from keras.callbacks import ReduceLROnPlateau, EarlyStopping
from tensorflow.keras.utils import to_categorical
from sklearn.utils import shuffle
from sklearn.feature_extraction.text import TfidfVectorizer

#Import important Libraries
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler  #library to normalize
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, roc_auc_score, auc, precision_recall_curve, roc_curve
from sklearn.linear_model import LogisticRegression   
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline
import pandas as pd                     # data processing, CSV file I/O (e.g. pd.read_csv)
import numpy as np                      # linear algebra
import seaborn as sns                   #  data visualization library 
from matplotlib import pyplot as plt    # plotting and visualization

In [5]:
df = pd.read_csv("F:/1.SecondSemester/Manufacturing/ManuData/AnotherCom/final_supervised_data_0_1_2.csv")
df.shape

(5938, 4)

In [6]:
nRow, nCol = df.shape                # shape give the dimensions of data in thr form of rows and columns
print(nRow)
print(nCol)

5938
4


# Pre-processing

In [7]:
df=pd.DataFrame(df)          
df_1=df.columns.tolist()              # This convert the column in the form of list.
print(df_1)    

['CNCSpindleLoad', 'AxisXLoad', 'AxisZLoad', 'cluster']


In [8]:
df.head(5)                      #It helps to visualized the dataset

,CNCSpindleLoad,AxisXLoad,AxisZLoad,cluster
0,41,29,4,0
1,24,52,6,1
2,12,16,5,1
3,39,32,8,0
4,39,30,11,0


In [9]:
df.isnull().sum()       # This function gives the information about Null values against each features

CNCSpindleLoad    0
AxisXLoad         0
AxisZLoad         0
cluster           0
dtype: int64

In [10]:
from sklearn.preprocessing import LabelEncoder

def labelencoder(df):                                     #It convert the catorigcal and string data into numerical values to 
    for c in df.columns:                                        #interperate easily.
        if df[c].dtype=='object': 
            df[c] = df[c].fillna('N')
            lbl = LabelEncoder()
            lbl.fit(list(df[c].values))
            df[c] = lbl.transform(df[c].values)
    return df

In [11]:
data1=labelencoder(df)                            #calling function of labelencoder
data1                                             #Visualization of data after performing labelencoder  technique

,CNCSpindleLoad,AxisXLoad,AxisZLoad,cluster
0,41,29,4,0
1,24,52,6,1
2,12,16,5,1
3,39,32,8,0
4,39,30,11,0
...,...,...,...,...
5933,0,37,2,1
5934,0,36,1,1
5935,0,37,1,1
5936,0,37,1,1


In [12]:
Labels=data1['cluster']                        # It take cluster column as a targeted label.
dataX=data1.drop('cluster',axis=1)             #It drop the cluster column and consider all other values .

# Handle Missing Values or NAN values

In [13]:
from sklearn.impute import SimpleImputer                                         #SimpleImputer  help to filed the missing 
                                                                                   # or non-values  using means or median.
imputer = SimpleImputer(missing_values = np.nan, strategy = 'mean')
imputer = imputer.fit(dataX)
X = imputer.transform(dataX)

# Feature Selection
* Bruta or RFE are used as a feature selection method, It select useful features and drop less useful features.

In [18]:
from sklearn.ensemble import RandomForestClassifier
model =RandomForestClassifier(max_depth=1) #For Boruta
from boruta import BorutaPy
feat_selector = BorutaPy(model, n_estimators='auto', verbose=1, random_state=101)
feat_selector.fit(X,Labels)
print(feat_selector.support_)  #Should we accept the featur
print(feat_selector.ranking_) #Rank 1 is the best
X_filtered1 = feat_selector.transform(X)  #Apply feature selection and return transformed data

Iteration: 1 / 100
Iteration: 2 / 100
Iteration: 3 / 100
Iteration: 4 / 100
Iteration: 5 / 100
Iteration: 6 / 100
Iteration: 7 / 100
Iteration: 8 / 100


BorutaPy finished running.

Iteration: 	9 / 100
Confirmed: 	3
Tentative: 	0
Rejected: 	0
[ True  True  True]
[1 1 1]


In [19]:
Final_selected=X_filtered1

In [20]:
from sklearn.preprocessing import MinMaxScaler

Final_selected =  (Final_selected - Final_selected.mean()) / Final_selected.std()
scaler = MinMaxScaler(feature_range=(0, 1))                        #It normalized the whole dataset between 0 and 1.
Final_selected = scaler.fit_transform(Final_selected)

In [21]:
#This step divided the whole dataset into training and testing by keeping test size 20%

X_train, X_test,y_train, y_test = train_test_split(Final_selected,Labels, test_size=0.20,random_state=9)

In [22]:
print('training features =',X_train.shape)
print('testing features   =',y_train.shape)
print('training labels=',X_test.shape)
print('testing labels =',y_test.shape)

training features = (4750, 3)
testing features   = (4750,)
training labels= (1188, 3)
testing labels = (1188,)


# RandomForest (RF)

In [23]:
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier(n_estimators=100)       
model.fit(X_train, y_train)
prediction = model.predict(X_test)

In [24]:
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
print(classification_report(y_test, prediction))
print(accuracy_score(y_test, prediction))
print(confusion_matrix(y_test, prediction))

              precision    recall  f1-score   support

           0       1.00      0.99      1.00       402
           1       1.00      1.00      1.00       757
           2       0.97      0.97      0.97        29

    accuracy                           1.00      1188
   macro avg       0.99      0.99      0.99      1188
weighted avg       1.00      1.00      1.00      1188

0.9966329966329966
[[399   2   1]
 [  0 757   0]
 [  1   0  28]]


# SVM

In [25]:
from sklearn.svm import SVC
from sklearn import svm
#model = svm.LinearSVC(multi_class="ovr")
model = svm.SVC(kernel='rbf', gamma=7.9, C=20, decision_function_shape='ovo')
model.fit(X_train, y_train)
prediction = model.predict(X_test)

In [26]:
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
print(classification_report(y_test, prediction))
print(accuracy_score(y_test, prediction))
print(confusion_matrix(y_test, prediction))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00       402
           1       1.00      1.00      1.00       757
           2       0.97      1.00      0.98        29

    accuracy                           1.00      1188
   macro avg       0.99      1.00      0.99      1188
weighted avg       1.00      1.00      1.00      1188

0.9983164983164983
[[400   1   1]
 [  0 757   0]
 [  0   0  29]]


# Decision Tree (DT)

In [27]:
from sklearn.tree import DecisionTreeClassifier
model = DecisionTreeClassifier()
model.fit(X_train, y_train)
prediction = model.predict(X_test)

In [28]:
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
print(classification_report(y_test, prediction))
print(accuracy_score(y_test, prediction))
print(confusion_matrix(y_test, prediction))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00       402
           1       1.00      1.00      1.00       757
           2       0.97      0.97      0.97        29

    accuracy                           1.00      1188
   macro avg       0.99      0.99      0.99      1188
weighted avg       1.00      1.00      1.00      1188

0.9966329966329966
[[400   1   1]
 [  1 756   0]
 [  1   0  28]]


# Gradient Boosting Machine(GBM)

In [29]:
from sklearn.ensemble import GradientBoostingClassifier
model = GradientBoostingClassifier(random_state=101)
model.fit(X_train, y_train)
prediction = model.predict(X_test)

In [30]:
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
print(classification_report(y_test, prediction))
print(accuracy_score(y_test, prediction))
print(confusion_matrix(y_test, prediction))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00       402
           1       1.00      1.00      1.00       757
           2       0.97      0.97      0.97        29

    accuracy                           1.00      1188
   macro avg       0.99      0.99      0.99      1188
weighted avg       1.00      1.00      1.00      1188

0.9974747474747475
[[400   1   1]
 [  0 757   0]
 [  1   0  28]]


# KNN

In [33]:
from sklearn.neighbors import KNeighborsClassifier
model = KNeighborsClassifier(n_neighbors=2)
model.fit(X_train, y_train)
prediction = model.predict(X_test)

C:\Users\Imtiyaj\anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)


In [34]:
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
print(classification_report(y_test, prediction))
print(accuracy_score(y_test, prediction))
print(confusion_matrix(y_test, prediction))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00       402
           1       1.00      1.00      1.00       757
           2       0.97      0.97      0.97        29

    accuracy                           1.00      1188
   macro avg       0.99      0.99      0.99      1188
weighted avg       1.00      1.00      1.00      1188

0.9966329966329966
[[400   1   1]
 [  1 756   0]
 [  1   0  28]]


# XGB

In [39]:
from xgboost import XGBClassifier
model = XGBClassifier()
model.fit(X_train, y_train)
prediction = model.predict(X_test)

In [40]:
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
print(classification_report(y_test, prediction))
print(accuracy_score(y_test, prediction))
print(confusion_matrix(y_test, prediction))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00       402
           1       1.00      1.00      1.00       757
           2       0.97      0.97      0.97        29

    accuracy                           1.00      1188
   macro avg       0.99      0.99      0.99      1188
weighted avg       1.00      1.00      1.00      1188

0.9966329966329966
[[400   1   1]
 [  1 756   0]
 [  1   0  28]]


# MLP

In [41]:
from sklearn.neural_network import MLPClassifier
MLP = MLPClassifier(max_iter=1500,activation='relu', learning_rate_init=0.001,shuffle=True,
                    learning_rate='constant', beta_1=0.999, beta_2=0.9 , momentum=0.88,
                    power_t=0.9, solver='lbfgs', alpha=1e-6, random_state=101)

In [42]:
MLP.fit(X_train, y_train)
prediction = MLP.predict(X_test)

In [43]:
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
print(classification_report(y_test, prediction))
print(accuracy_score(y_test, prediction))
print(confusion_matrix(y_test, prediction))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00       402
           1       1.00      1.00      1.00       757
           2       0.97      0.97      0.97        29

    accuracy                           1.00      1188
   macro avg       0.99      0.99      0.99      1188
weighted avg       1.00      1.00      1.00      1188

0.9983164983164983
[[401   0   1]
 [  0 757   0]
 [  1   0  28]]
